### Grey Converter

In [57]:
import cv2

def rgb_to_grey(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return grey

2. Given any two images, write a function to compute the quality metrics PSNR and MSE, where I1 refers to the original image. I2 refers to the reconstructed (enhanced) image and M, N denotes the size of the image; L refers to the intensity levels defined in the give image (for a grey scale image L = 256). 

In [58]:
import math
import numpy as np

def find_mse(dt1, dt2):
    mse = np.sum((dt1-dt2)**2)/dt1.shape[0]*dt1.shape[1]
    return mse

def find_psnr(dt1, dt2):
    L = 256
    mse = find_mse(dt1,dt2)
    psnr = 20*math.log((L-1)/mse)
    return psnr

In [59]:
image = cv2.imread("AthulMathew40.jpg")
dt1 = rgb_to_grey(image)

image = cv2.imread("AthulMathew10.jpg")
dt2 = rgb_to_grey(image)

print("MSE", find_mse(dt1, dt2))
print("PSNR", find_psnr(dt1, dt2))

MSE 2753685.0
PSNR -185.74374054971116


3. Create a synthetic image with the following features and store it as “YourName.jpg”:          (256 x 256)

3a. 10% of pixels with values 50

In [60]:
import numpy as np
from PIL import Image
arr = np.random.choice([0, 50], size=256*256, p=[.9, .1])
array = np.reshape(arr, (256, 256))

# l - Greyscale Mode
data = Image.fromarray(array,'L')
data.save('AthulMathew10.jpg')
# data.show()

3b. 20% with 100

In [61]:
import numpy as np
from PIL import Image

arr = np.random.choice([0, 100], size=256*256, p=[.8, .2])
array = np.reshape(arr, (256, 256))

# l - Greyscale Mode
data = Image.fromarray(array,'L')
data.save('AthulMathew20.jpg')
# data.show()

3c. 30% with 150

In [62]:
import numpy as np
from PIL import Image

arr = np.random.choice([0, 150], size=256*256, p=[.7, .3])
array = np.reshape(arr, (256, 256))

# l - Greyscale Mode
data = Image.fromarray(array,'L')
data.save('AthulMathew30.jpg')
# data.show()

3d. 40% with 225

In [63]:
import numpy as np
from PIL import Image

arr = np.random.choice([0, 225], size=256*256, p=[.6, .4])
array = np.reshape(arr, (256, 256))

# l - Greyscale Mode
data = Image.fromarray(array,'L')
data.save('AthulMathew40.jpg')
# data.show()

3(i) Perform Log transform, gamma transform and histogram equalization 

### Log Transform

In [64]:
import numpy as np

def log_trans(dt):
    c = 255 / np.log(1 + np.max(dt)) 
    log_img = c * (np.log(dt + 1))
    log_img = np.array(log_img, dtype = np.uint8) 
    return log_img

Test:

In [65]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

log_img = log_trans(dt)
dt = Image.fromarray(log_img,'L')
# dt.show()

<ipython-input-64-559cca4ad191>:5: RuntimeWarning: divide by zero encountered in log
  log_img = c * (np.log(dt + 1))


### Histogram Equalization

In [66]:
import cv2
import numpy as np
import collections

# def hist_eq(image1):
#     gray_img = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)

#     equ = cv2.equalizeHist(gray_img) 
#     return equ

def hist_eq(dt):
    lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
        
            lst.append(dt[i][j])

    count = []
    cum_sum = []
    summ = 0
    for i in range(256):
        pixel_count = lst.count(i)
        count.append(pixel_count)
        summ =summ + pixel_count
        cum_sum.append(summ)
    
    h_v = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            hv = int(round(((cum_sum[dt[i][j]]-1)*255)/(dt.shape[0]*dt.shape[1]),0))
            h_v.append(hv)
    
    
    array = np.reshape(h_v, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8) 
    return array

Test:

In [67]:
import cv2
from matplotlib import pyplot as plt

# image1 = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")

# equ = hist_eq(image1)

# plt.imshow(equ, cmap = 'gray')
# plt.show()

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

hist_img = hist_eq(dt)
dt = Image.fromarray(hist_img,'L')
# dt.show()

### Gamma Transformation

In [68]:
def gamma_eq(dt, gamma, c_slider):
    gamma_img = pow((c_slider*dt), gamma)
    array = np.array(gamma_img, dtype = np.uint8)
    return array

Test:

In [69]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

gamma = 20
c=1

gamma_img = gamma_eq(dt_in, gamma, c)

dt_out = Image.fromarray(gamma_img,'L')
# dt_out.show()

3(ii). Compute quality metrics and give inference on the resultant image.

In [70]:
print("MSE", find_mse(dt_in, dt_out))
print("PSNR", find_psnr(dt_in, dt_out))

MSE 1855764.0
PSNR -177.85086967825077


3(iii). Perform average filter, weighted average filter and show which is better (choose any filter size )

### Average Filter

In [71]:
import numpy as np
from PIL import Image

def find_avg(dt_in):
    
    filter_h, filter_w = 3, 3

    filter_coeff = [[1, 1, 1], 
                    [1, 1, 1], 
                    [1, 1, 1]]

    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt_in.shape[0]+ (2 * pad_h))
    data_w = int(dt_in.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt_in

    out_lst = []
    for i in range(dt_in.shape[0]):
        for j in range(dt_in.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(9))

    array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array


Test:

In [72]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

array = find_avg(dt_in)
dt_out1 = Image.fromarray(array,'L')
# dt_out.show()

In [73]:
print("MSE Average Filter ", find_mse(dt_in, dt_out1))
print("PSNR Average Filter ", find_psnr(dt_in, dt_out1))

MSE Average Filter  514752.0
PSNR Average Filter  -152.2035393021063


### Weighted Average Filter

In [74]:
import numpy as np
from PIL import Image

def find_wg_avg(dt_in):
    
    filter_h, filter_w = 3, 3

    filter_coeff = [[1, 2, 1], 
                    [2, 4, 2], 
                    [1, 2, 1]]

    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt_in.shape[0]+ (2 * pad_h))
    data_w = int(dt_in.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt_in

    out_lst = []
    for i in range(dt_in.shape[0]):
        for j in range(dt_in.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(16))

    array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array

Test:

In [75]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

array = find_wg_avg(dt_in)
dt_out2 = Image.fromarray(array,'L')
# dt_out.show()

In [76]:
print("MSE Weighted Average Filter ", find_mse(dt_in, dt_out2))
print("PSNR Weighted Average Filter ", find_psnr(dt_in, dt_out2))

MSE Weighted Average Filter  412827.0
PSNR Weighted Average Filter  -147.7904070565773


Average Filter has low mse. So it is better than Weighted Average Filter.

3(iv). Perform image sharpening.

In [77]:
def find_sharp(image1, sharp_size, filter_coeff):
    
    x = filter_coeff.split(",")
    x = [int(i) for i in x]
    x = np.array(x)

    if len(x) == 9:
        filter_coeff = x.reshape(3, 3)
    else:
        filter_coeff = x.reshape(5, 5)
    
    filter_h, filter_w = sharp_size, sharp_size
    
    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt.shape[0]+ (2 * pad_h))
    data_w = int(dt.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt

    out_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(filter_h*filter_w))



    array = np.reshape(out_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)

    return array

Test:

In [78]:
import numpy as np
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

filter_coeff = '0,-1,0,-1,4,-1,0,-1,0'

sharp_size = 3

array = find_sharp(dt, sharp_size, filter_coeff)
sh_dt = Image.fromarray(array,'L')
# sh_dt.show()

4. Create a synthetic image with the following features and store as “MyRegno.jpg” (256x256)

First 20% of the rows (or columns) to have intensity value as 10, Next 20% as 30, Next 20% as 50, Next 20% as 100, Other rows with values as 150 

In [79]:
import numpy as np
from PIL import Image

arr = np.zeros(256*256)
array = np.reshape(arr, (256, 256))

array[0:2*int(256/10)] = 10
array[2*int(256/10):4*int(256/10)] = 30
array[4*int(256/10):6*int(256/10)] = 50
array[6*int(256/10):8*int(256/10)] = 100
array[8*int(256/10):] = 150

# l - Greyscale Mode
array = np.array(array, dtype = np.uint8)
data = Image.fromarray(array,'L')
data.save('20016.jpg')
print(array)

# data.show()

[[ 10  10  10 ...  10  10  10]
 [ 10  10  10 ...  10  10  10]
 [ 10  10  10 ...  10  10  10]
 ...
 [150 150 150 ... 150 150 150]
 [150 150 150 ... 150 150 150]
 [150 150 150 ... 150 150 150]]


4(i). Perform transformation using g(x,y) = f(x,y) + c * log(x,y)

In [80]:
import math

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

c = .01
g_list = []
for i in range(dt_in.shape[0]):
    for j in range(dt_in.shape[1]):
        if dt_in[i][j] != 0 :
            g_list.append(dt_in[i][j] + c*(math.log(dt_in[i][j])))
        else:
            g_list.append(dt_in[i][j])
    
array = np.reshape(g_list, (dt_in.shape[0], dt_in.shape[1]))
array = np.array(array, dtype = np.uint8)
dt_out1 = Image.fromarray(array,'L')
# dt_out1.show()

4(ii). Perform transformation using g(x,y)=f(x,y) – c * log(x,y)

In [81]:
import math

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

c = .01
g_list = []
for i in range(dt_in.shape[0]):
    for j in range(dt_in.shape[1]):
        if dt_in[i][j] != 0 :
            g_list.append(dt_in[i][j] - c*(math.log(dt_in[i][j])))
        else:
            g_list.append(dt_in[i][j])
    
array = np.reshape(g_list, (dt_in.shape[0], dt_in.shape[1]))
array = np.array(array, dtype = np.uint8)
dt_out2 = Image.fromarray(array,'L')
# dt_out2.show()

4(iii). Compare images generated by the above two transformations and conclude which transformation function is better for a given value of c.

In [82]:
print("MSE + log", find_mse(dt_in, dt_out1))
print("PSNR + log", find_psnr(dt_in, dt_out1))

print("MSE - log", find_mse(dt_in, dt_out2))
print("PSNR - log", find_psnr(dt_in, dt_out2))

MSE + log 0.0
PSNR + log inf
MSE - log 21580.0
PSNR - log -88.76517387307399


<ipython-input-58-96ab444548b6>:11: RuntimeWarning: divide by zero encountered in double_scalars
  psnr = 20*math.log((L-1)/mse)


Both Tranformtaions are same for this Image with C=0.01

4(iv). Apply smoothing filter and show the resultant image, also compute metrics to show the quality 

In [83]:
import numpy as np
from PIL import Image

filter_h, filter_w = 3, 3

filter_coeff = [[1, 1, 1], 
                [1, 1, 1], 
                [1, 1, 1]]

pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

# maxx = dt.shape[0] if dt.shape[0]>dt.shape[1] else dt.shape[1]
# print(dt.shape[0],dt.shape[1])

# im_rez = im.resize((maxx, maxx))
# dt = np.asarray(im_rez)

data_h = int(dt_in.shape[0]+ (2 * pad_h))
data_w = int(dt_in.shape[1] + (2 * pad_w))

data = np.zeros([data_h , data_w])
 
data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt

out_lst = []
for i in range(dt_in.shape[0]):
    for j in range(dt_in.shape[1]):
        out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(filter_h*filter_w))



array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
array = np.array(array, dtype = np.uint8)
dt_out = Image.fromarray(array,'L')
# dt_out.show()

In [84]:
print("MSE - smooth", find_mse(dt_in, dt_out))
print("PSNR - smooth", find_psnr(dt_in, dt_out))

MSE - smooth 514752.0
PSNR - smooth -152.2035393021063


5. Write a program to allow user to choose any grey scale image from your data set and do the following:

5a. Perform Gamma transform by accepting “gamma” value from user, the transformation is given by g(x,y) = cα f(x,y) -  accept parameter values from user.

In [85]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

gamma = 20
c=1

gamma_img = gamma_eq(dt_in, gamma, c)

dt_out1 = Image.fromarray(gamma_img,'L')
# dt_out1.show()

5b. Perform histogram equalization 

In [86]:
image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

hist_img = hist_eq(dt_in)
dt_out2 = Image.fromarray(hist_img,'L')
# dt_out2.show()

5c. Show the resultant images after a and b, and infer which produces better quality image

In [87]:
print("MSE gamma", find_mse(dt_in, dt_out1))
print("PSNR gamma", find_psnr(dt_in, dt_out1))

print("MSE hist", find_mse(dt_in, dt_out2))
print("PSNR hist", find_psnr(dt_in, dt_out2))

MSE gamma 1855764.0
PSNR gamma -177.85086967825077
MSE hist 2017929.0
PSNR hist -179.5263750153693


### Median Filtering

6a. Perform Median filter and show the resultant image

In [122]:
def fil_med(dt):
    med_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):

            if (i != 0 and i < dt.shape[0]-1) and (j != 0 and j< dt.shape[1]-1):

                ngh_lst = [ dt[i-1][j-1], dt[i][j-1], dt[i+1][j-1], 
                           dt[i+1][j], dt[i+1][j+1], dt[i][j+1], 
                           dt[i-1][j+1], dt[i-1][j] ]

                median = statistics.median(ngh_lst)
                med_lst.append(median)

            else:
                med_lst.append(dt[i][j])

    array = np.reshape(med_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)
    return array

Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Filtering Gamma 0 0 Weighted Avg. Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0


Test:

In [89]:
import numpy as np
import statistics 
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

array = fil_med(dt)
dt_med = Image.fromarray(array,'L')
# dt_med.show()

C:\Users\athul\anaconda3\lib\statistics.py:435: RuntimeWarning: overflow encountered in ubyte_scalars
  return (data[i - 1] + data[i])/2


### Mode Filtering

6b. Perform Mode filter and show the resultant image

In [90]:
def fil_mod(dt):
    mod_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):

            if (i != 0 and i < dt.shape[0]-1) and (j != 0 and j< dt.shape[1]-1):

                ngh_lst = [ dt[i-1][j-1], dt[i][j-1], dt[i+1][j-1], 
                           dt[i+1][j], dt[i+1][j+1], dt[i][j+1], 
                           dt[i-1][j+1], dt[i-1][j] ]

                mode = statistics.mode(ngh_lst)
                mod_lst.append(mode)

            else:
                mod_lst.append(dt[i][j])

    array = np.reshape(mod_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)
    return array

Test:

In [91]:
import numpy as np
import statistics 
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

array = fil_mod(dt)
dt_mod = Image.fromarray(array,'L')
# dt_mod.show()

6c. Show the resultant images and infer which produces better quality image

In [92]:
print("MSE Median Filter", find_mse(dt, dt_med))
print("PSNR Median Filter", find_psnr(dt, dt_med))

print("MSE Mode Filter", find_mse(dt, dt_mod))
print("PSNR Mode Filter", find_psnr(dt, dt_mod))

MSE Median Filter 399206.0
PSNR Median Filter -147.11938616416379
MSE Mode Filter 928419.9999999999
PSNR Mode Filter -163.99951900861123


Mode filtering gave better output

### Statistics

7a. Compute statistical measures: Mean, Mode, Standard deviation and Median of the chosen image

In [93]:
def find_statistic(dt):
    count = dt.shape[0]*dt.shape[0]

    summ = 0
    pixel_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            summ = summ + dt[i][j]
            pixel_lst.append(dt[i][j])

    mean = summ/count  

    var_sum = 0
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            var_sum = var_sum + (dt[i][j]-mean)**2

    variance = var_sum/(count-1)
    sd = variance**0.5
    mode = statistics.mode(pixel_lst)
    median = statistics.median (pixel_lst)

    strr = " "
    print("Sum", summ)
    strr = strr + "Sum: " + str(summ) + ","
    print("Count", count)
    strr = strr + " Count: " + str(count) + ","
    print("Mean", mean)
    strr = strr + " Mean: " + str(mean) + ","
    print("Mode", mode)
    strr = strr + " Mode: " + str(mode) + ","
    print("Median", median)
    strr = strr + " Median: " + str(median) + ","
    print("Variance", variance)
    strr = strr + " Variance: " + str(variance) + ","
    print("Standarad Deviation", sd)
    strr = strr + " Standarad Deviation: " + str(sd) + ";"

    return strr

Test:

In [94]:
import numpy as np
import statistics 
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

find_statistic(dt)

Sum 3728113
Count 22500
Mean 165.6939111111111
Mode 0
Median 61.0
Variance 4649.026488547031
Standarad Deviation 68.18376997898423


' Sum: 3728113, Count: 22500, Mean: 165.6939111111111, Mode: 0, Median: 61.0, Variance: 4649.026488547031, Standarad Deviation: 68.18376997898423;'

### BitPlanes

7b. Represent the image as individual bit planes and show the planes as individual images.

In [95]:
def get_bitplanes(dt, show_planes):
    lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            lst.append(dt[i][j])


    lst = [ np.binary_repr(i, width=8) for i in lst]
    bit_img = []

    eight_bit_img = (np.array([int(i[0]) for i in lst]) * 128).reshape(dt.shape[0],dt.shape[1])
    seven_bit_img = (np.array([int(i[1]) for i in lst]) * 64).reshape(dt.shape[0],dt.shape[1])
    six_bit_img = (np.array([int(i[2]) for i in lst]) * 32).reshape(dt.shape[0],dt.shape[1])
    five_bit_img = (np.array([int(i[3]) for i in lst]) * 16).reshape(dt.shape[0],dt.shape[1])
    four_bit_img = (np.array([int(i[4]) for i in lst]) * 8).reshape(dt.shape[0],dt.shape[1])
    three_bit_img = (np.array([int(i[5]) for i in lst]) * 4).reshape(dt.shape[0],dt.shape[1])
    two_bit_img = (np.array([int(i[6]) for i in lst]) * 2).reshape(dt.shape[0],dt.shape[1])
    one_bit_img = (np.array([int(i[7]) for i in lst]) * 1).reshape(dt.shape[0],dt.shape[1])
    
    bit_img.append(eight_bit_img)
    bit_img.append(seven_bit_img)
    bit_img.append(six_bit_img)
    bit_img.append(five_bit_img)
    bit_img.append(four_bit_img)
    bit_img.append(three_bit_img)
    bit_img.append(two_bit_img)
    bit_img.append(one_bit_img)
    

    array = np.reshape(bit_img[8-show_planes], (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array, bit_img

Test:

In [96]:
import numpy as np
import statistics 
from PIL import Image

image = cv2.imread("image.jpg")
dt = rgb_to_grey(image)

show_planes = 8

array, bit_img = get_bitplanes(dt, show_planes)
bit_dt = Image.fromarray(array,'L')

# bit_dt.show()

### Reconstruct Bit-Planes

7c. Reconstruct the image with 3 most significant bit planes.

In [97]:
def get_bitrecon(dt, bit_planes):
    
    array, bit_img = get_bitplanes(dt, 8)
    
    arr = np.zeros(dt.shape[0]*dt.shape[1])
    img_comb = np.reshape(arr, (dt.shape[0], dt.shape[1]))
    
    for i in range(bit_planes):
        img_comb = img_comb + bit_img[i]

    array = np.reshape(img_comb, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)
    return array

Test:

In [98]:
image = cv2.imread("image.jpg")
dt = rgb_to_grey(image)

bit_planes = 1

array = get_bitrecon(dt, bit_planes)
comb_dt = Image.fromarray(array,'L')
# comb_dt.show()

7d. Compute quality metrics to estimate the quality of the resultant image.

In [99]:
print("MSE Median Filter", find_mse(dt, comb_dt))
print("PSNR Median Filter", find_psnr(dt, comb_dt))

MSE Median Filter 2260173.0
PSNR Median Filter -181.79376743665816


### Smoothing

8a. Smoothing by a choosing filter based on user choice (size and coefficients)

    Smoothing Spatial Filters • Smoothing filters are used for noise reduction and blurring operations. • It takes into account the pixels surrounding it in order to make a determination of a more accurate version of this pixel. • By taking neighboring pixels into consideration, extreme “noisy” pixels can be filtered out. 

In [100]:
def find_smooth(dt, smooth_size, filter_coeff):
    x = filter_coeff.split(",")
    x = [int(i) for i in x]
    x = np.array(x)

    if len(x) == 9:
        filter_coeff = x.reshape(3, 3)
    else:
        filter_coeff = x.reshape(5, 5)
    
    filter_h, filter_w = smooth_size, smooth_size

    
    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)
    
    data_h = int(dt.shape[0]+ (2 * pad_h))
    data_w = int(dt.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt

    out_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(filter_h*filter_w))



    array = np.reshape(out_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)

    return array

Test:

In [101]:
import numpy as np
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

filter_coeff = '1,1,1,1,1,1,1,1,1'

smooth_size = 3

array = find_smooth(dt, smooth_size, filter_coeff)
sm_dt = Image.fromarray(array,'L')
# sm_dt.show()

### Sharpening

8b. Sharpen by a user defined filter 

In [120]:
def find_sharp(dt, sharp_size, filter_coeff):
    
    x = filter_coeff.split(",")
    x = [int(i) for i in x]
    x = np.array(x)

    if len(x) == 9:
        filter_coeff = x.reshape(3, 3)
    else:
        filter_coeff = x.reshape(5, 5)
    
    filter_h, filter_w = sharp_size, sharp_size
    
    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt.shape[0]+ (2 * pad_h))
    data_w = int(dt.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt

    out_lst = []
    for i in range(dt.shape[0]):
        for j in range(dt.shape[1]):
            out_lst.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeff)/(filter_h*filter_w))



    array = np.reshape(out_lst, (dt.shape[0], dt.shape[1]))
    array = np.array(array, dtype = np.uint8)

    return array

Sharpening Gamma 0 0 Median Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sharpening Gamma 0 0 Median Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sharpening Gamma 0 0 Median Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sharpening Gamma 0 0 Median Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sharpening Gamma 0 0 Median Show Statistics 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0


In [103]:
import numpy as np
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt = rgb_to_grey(image)

filter_coeff = '0,-1,0,-1,4,-1,0,-1,0'

sharp_size = 3

array = find_sharp(dt, sharp_size, filter_coeff)
sh_dt = Image.fromarray(array,'L')
# sh_dt.show()

8c. Apply Roberts or Sobel filter and show the result

### Sobels Filter

In [104]:
def get_sobel(dt_in):
    filter_h, filter_w = 3, 3

    filter_coeffx = [[-1, 0, 1], 
                    [-2, 0, 2], 
                    [-1, 0, 1]]

    filter_coeffy = [[-1, -2, -1], 
                    [0, 0, 0], 
                    [1, 2, 1]]

    pad_h, pad_w = int((filter_h-1)/2), int((filter_w-1)/2)

    data_h = int(dt_in.shape[0]+ (2 * pad_h))
    data_w = int(dt_in.shape[1] + (2 * pad_w))

    data = np.zeros([data_h , data_w])

    data[pad_h:int(data_h - pad_h), pad_w:int(data_w - pad_w)] = dt_in

    out_lst = []
    out_lstx = []
    out_lsty = []
    for i in range(dt_in.shape[0]):
        for j in range(dt_in.shape[1]):
            out_lstx.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeffx)/(filter_h*filter_w))
            out_lsty.append(np.sum((data[i:i + filter_h, j:j + filter_w])*filter_coeffy)/(filter_h*filter_w))

    out_lst = [ abs(i) + abs(j) for i,j in zip(out_lstx, out_lsty) ]

    array = np.reshape(out_lst, (dt_in.shape[0], dt_in.shape[1]))
    array = np.array(array, dtype = np.uint8)
    
    return array

Test

In [105]:
import numpy as np
from PIL import Image

image = cv2.imread("C:\\Users\\athul\\3.ComputerVision\\Images\\14_std.jpg")
dt_in = rgb_to_grey(image)

array = get_sobel(dt_in)
dt_sb = Image.fromarray(array,'L')
# dt_sb.show()

8d. Compare which filter is better for the chosen image for smoothing and sharpening using quality metrics. 

In [106]:
print("MSE Sobel Filter", find_mse(dt_in, dt_sb))
print("PSNR Sobel Filter", find_psnr(dt_in, dt_sb))

MSE Sobel Filter 2360566.0
PSNR Sobel Filter -182.66296867194202


9. Generate a synthetic image with four concentric squares of 4 intensity values (40, 80, 160, 220). Apply filters based on user choice and show the resultant images and your inference. 

In [107]:
import numpy as np
from PIL import Image

arr = np.zeros(256*256)
array = np.reshape(arr, (256, 256))

array[0:5*int(256/10)][0:5*int(256/10)] = 40
array[0:5*int(256/10)][5*int(256/10):] = 80
array[5*int(256/10):][0:5*int(256/10)] = 160
array[5*int(256/10):][5*int(256/10):] = 220


# l - Greyscale Mode
array = np.array(array, dtype = np.uint8)
data = Image.fromarray(array,'L')
data.save('synth.jpg')
print(array)

# data.show()

[[ 40  40  40 ...  40  40  40]
 [ 40  40  40 ...  40  40  40]
 [ 40  40  40 ...  40  40  40]
 ...
 [220 220 220 ... 220 220 220]
 [220 220 220 ... 220 220 220]
 [220 220 220 ... 220 220 220]]


In [108]:
image = cv2.imread("synth.jpg")
dt = rgb_to_grey(image)

In [109]:
array = fil_mod(dt)
dt_mod = Image.fromarray(array,'L')
# dt_mod.show()

print("MSE Mode Filter", find_mse(dt, dt_mod))
print("PSNR Mode Filter", find_psnr(dt, dt_mod))

MSE Mode Filter 46736.0
PSNR Mode Filter -104.22012958938191


In [110]:
array = fil_med(dt)
dt_med = Image.fromarray(array,'L')
# dt_med.show()

print("MSE Median Filter", find_mse(dt, dt_med))
print("PSNR Median Filter", find_psnr(dt, dt_med))

MSE Median Filter 14224.0
PSNR Median Filter -80.4284482519052


Median Filter is better since mse value is less compared to mode filtering.

In [149]:
def choice(image1, selector, transformation, gamma, 
              c_slider, filtering, show_planes, 
              bit_planes, smoothing, smooth_size, filter_coeff_sm,
              sharpening, sharp_size, filter_coeff_sh):
    
    print(selector, transformation, gamma, 
              c_slider, filtering, 
              statistic, show_planes, 
              bit_planes, smoothing, smooth_size, filter_coeff_sm,
              sharpening, sharp_size, filter_coeff_sh)
    
    image2 = image1
    
    stat_out = " "
    
    if selector == "Transformations":
        if transformation == "Histogram Equalization":
            
            hist_img = hist_eq(rgb_to_grey(image1))
            image2 = Image.fromarray(hist_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
            
        elif transformation == "Gamma":
            gamma_img = gamma_eq(rgb_to_grey(image1), gamma, c_slider)
            image2 = Image.fromarray(gamma_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
    
    elif selector == "Filtering":
        if filtering == "Median":
            med_img = fil_med(rgb_to_grey(image1))
            image2 = Image.fromarray(med_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
            
        elif filtering == "Mode":
            mod_img = fil_mod(rgb_to_grey(image1))
            image2 = Image.fromarray(mod_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
            
        elif filtering == "Average":
            mod_img = find_avg(rgb_to_grey(image1))
            image2 = Image.fromarray(mod_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
            
        elif filtering == "Weighted Avg.":
            mod_img = find_wg_avg(rgb_to_grey(image1))
            image2 = Image.fromarray(mod_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
            
    elif selector == "Statistical Measures":
        stat_out = find_statistic(rgb_to_grey(image1))
        image2 = image1

    elif selector == "Bit Planes":
        bit_pl, bit_img = get_bitplanes(rgb_to_grey(image1), show_planes)
        image2 = Image.fromarray(bit_pl,'L')
#         stat_out = find_statistic(rgb_to_grey(image1))
            
    elif selector == "Reconstruct Bit Planes":
        bit_recon = get_bitrecon(rgb_to_grey(image1), bit_planes)
        image2 = Image.fromarray(bit_recon,'L')
#         stat_out = find_statistic(rgb_to_grey(image1))
            
    elif selector == "Smoothing":
        if smoothing == " ":
            if smooth_size == "3x3":
                smooth_size = 3
            else:
                smooth_size = 5
                
            smoth_img = find_smooth(rgb_to_grey(image1), smooth_size, filter_coeff_sm)
            image2 = Image.fromarray(smoth_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))
        
    elif selector == "Sharpening":
        if sharpening == " ":
            if sharp_size == "3x3":
                sharp_size = 3
            else:
                sharp_size = 5
                
            sh_img = find_sharp(rgb_to_grey(image1), sharp_size, filter_coeff_sh)
            image2 = Image.fromarray(sh_img,'L')
#             stat_out = find_statistic(rgb_to_grey(image1))

    elif selector == "Edge Detection - Sobels":
        sob_img = get_sobel(rgb_to_grey(image1))
        image2 = Image.fromarray(sob_img,'L')
#         stat_out = find_statistic(rgb_to_grey(image1))
  
    return image2, stat_out

In [150]:
import gradio as gr

img = gr.inputs.Image()

title = "Computer Vision WorkBench"
# description = "An experimental UI to demostrate Image Processing."

selector = gr.inputs.Dropdown(["Transformations", "Filtering", 
                               "Statistical Measures", "Bit Planes", 
                               "Reconstruct Bit Planes",
                               "Smoothing", "Sharpening", 
                               "Edge Detection - Sobels"], label="Select Operation")

transformation = gr.inputs.Radio(['Gamma', 'Histogram Equalization'], label="Transformations")
gamma = gr.inputs.Slider(minimum=0, maximum = 100, step=0.1, default = 2, label="Select Gamma")
c_slider = gr.inputs.Slider(minimum=0, maximum = 100, step=1, default = 5, label="Select C Value")

filtering = gr.inputs.Radio(['Median', 'Mode', "Average", "Weighted Avg."], label="Filtering")
# statistic = gr.inputs.Radio(['Show Statistics'], label="Statistical Measures")

show_planes = gr.inputs.Slider(minimum=1, maximum = 8, step=1, default = 8, label="Bit Planes")
bit_planes = gr.inputs.Slider(minimum=1, maximum = 8, step=1, default = 3, label="Reconstruct using Bit Plane")

smoothing = gr.inputs.Radio([" "], label="Smoothing")
dataset = ['3x3', '5x5']
smooth_size = gr.inputs.Dropdown(dataset, label="Select Filter size")
# smooth_size = gr.inputs.Slider(minimum=0, maximum = 10, step=1, default = 3, label="Filter Size")
filter_coeff_sm = gr.inputs.Textbox(lines=2,default= [1, 1, 1, 1, 1, 1 ,1 ,1, 1], numeric=False, type="str", label=None)

sharpening = gr.inputs.Radio([" "], label="Sharpening")
# sharp_size = gr.inputs.Slider(minimum=0, maximum = 10, step=1, default = 3, label="Filter Size")
sharp_size = gr.inputs.Dropdown(dataset, label="Select Filter size")
filter_coeff_sh = gr.inputs.Textbox(lines=2,default= [0, -1, 0, -1, 4, -1, 0, -1, 0], numeric=False, type="str", label=None)

image1 = gr.inputs.Image(shape=(299, 299), type="numpy")
image2 = gr.outputs.Image(type="numpy", label="Output")

stat_out = gr.outputs.Textbox(type="auto", label="Statistics")

gr.Interface(choice, inputs = [image1, selector, transformation, gamma, c_slider, filtering, 
                                  show_planes, bit_planes, smoothing, smooth_size, filter_coeff_sm,
                                  sharpening, sharp_size, filter_coeff_sh],   
             outputs = [image2, stat_out], 
             title=title, live=True).launch(share=True)

IMPORTANT: You are using gradio version 1.4.0, however version 1.5.0 is available, please upgrade.
--------
Running locally at: http://127.0.0.1:7883/
This share link will expire in 24 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://35995.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7883/',
 'https://35995.gradio.app')

Statistical Measures Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sum 10725682
Count 89401
Mean 119.9727296115256
Mode 8
Median 140
Variance 5439.043909560909
Standarad Deviation 73.74987396301711
Statistical Measures Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sum 10725682
Count 89401
Mean 119.9727296115256
Mode 8
Median 140
Variance 5439.043909560909
Standarad Deviation 73.74987396301711
Statistical Measures Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sum 10725682
Count 89401
Mean 119.9727296115256
Mode 8
Median 140
Variance 5439.043909560909
Standarad Deviation 73.74987396301711
Statistical Measures Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Sum 10725682
Count 89401
Me

Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 10 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 11 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 11 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 12 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 12 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 12 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0


Transformations Gamma 6.8 14 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 14 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 14 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 14 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 14 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 15 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 17 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0

...

Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 6.8 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transf

Transformations Gamma 1.2 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 1.2 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 1.2 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 1.9 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 2.6 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 3.6 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 5.1 9 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transf

Transformations Gamma 4.3 31 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 30 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 24 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 17 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 15 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 13 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 10 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0

Transformations Gamma 4.3 1 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transf

Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Transformations Gamma 4.3 2 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Edge Detection - Sobels Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Edge Detection - Sobels Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,4,-1,0,-1,0
Edge Detection - Sobels Gamma 0 0 Median <gradio.inputs.Radio object at 0x000001A54D4FA280> 1 1   3x3 1,1,1,1,1,1,1,1,1   3x3 0,-1,0,-1,